In [77]:
import pandas as pandas
import numpy as numpy
import os
import matplotlib.pyplot as matplot
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import itertools
from IPython.display import Image  
from sklearn import tree
from os import system
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.grid_search import GridSearchCV
numpy.random.seed(1234)
RandomState = numpy.random.seed(1234)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')


# In[8]:


class Perform_EDA():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        
    def EDA_Corr(df):
        corr = df.corr()
        index = corr.columns
        Output = []
        for i in range(0,len(index)):
            i = index[i]
            Pos = corr.index[(corr[i] >= 0.5)].tolist()
            No = corr.index[(corr[i] < 0.5) & (corr[i] > -0.6)].tolist()
            Neg = corr.index[(corr[i] <= -0.5)].tolist()
            leng_u = len(No)
            leng_pos = len(Pos)
            leng_neg = len(Neg)
            Out = [i, leng_u, leng_pos, leng_neg, Pos, Neg, No]
            Output.append(Out)
        fig, ax = matplot.subplots(figsize=(20,10))  
        sns.heatmap(corr,annot=True,vmin=-1,vmax=1,cmap='Blues', linewidths=0, ax = ax)
        Output1 = pandas.DataFrame(Output, columns= ['Feature','Uniqueness','Positive rel', 'inverse rel', 'Pos', 'Neg', 'No'])
        return Output1

    def EDA(df):
        EDA = pandas.DataFrame((df.describe()).T)
        EDA["Kurtosis"] = df.kurtosis()
        EDA["Skewness"] = df.skew()
        EDA["Range"] = EDA['max'] -  EDA['min']
        EDA["IQR"] = EDA['75%'] -  EDA['25%']
        EDA["Missing Values"] = df.shape[0] - EDA["count"]
        print("Total Missing Values = ", EDA['Missing Values'].sum(), "Data Points, Contributing to ", 
              (round(((EDA['Missing Values'].sum())/len(df)),2))*100,"%")
        print("Columns with values as 0 \n\n",pandas.Series((EDA.loc[EDA['min'] == 0]).index))
        return EDA
    
    def NaN_treatment_median(df):
        EDA_Summary = Perform_EDA.EDA(df)
        Missing_Value_Columns = pandas.Series((EDA_Summary.loc[EDA_Summary['Missing Values'] != 0]).index)
        len(Missing_Value_Columns)
        for i in range(0,len(Missing_Value_Columns)):
            df[Missing_Value_Columns[i]].fillna(df[Missing_Value_Columns[i]].median(), inplace = True)
        return df
    
    def NaN_treatment_mean(df):
        EDA_Summary = Perform_EDA.EDA(df)
        Missing_Value_Columns = pandas.Series((EDA_Summary.loc[EDA_Summary['Missing Values'] != 0]).index)
        len(Missing_Value_Columns)
        for i in range(0,len(Missing_Value_Columns)):
            df[Missing_Value_Columns[i]].fillna(df[Missing_Value_Columns[i]].mean(), inplace = True)
        return df
    
    def Zero_Values_Treatment_Median(df):
        EDA_Summary = Perform_EDA.EDA(df)
        Zero_Value_Columns = pandas.Series((EDA_Summary.loc[EDA_Summary['min'] == 0]).index)
        for i in range(0,len(Zero_Value_Columns)):
            df[Zero_Value_Columns[i]].replace(0,(df[Zero_Value_Columns[i]]).median(), inplace = True)
        return df
    
    def Zero_Values_Treatment_Mean(df):
        EDA_Summary = Perform_EDA.EDA(df)
        Zero_Value_Columns = pandas.Series((EDA_Summary.loc[EDA_Summary['min'] == 0]).index)
        for i in range(0,len(Zero_Value_Columns)):
            df[Zero_Value_Columns[i]].replace(0,(df[Zero_Value_Columns[i]]).mean(), inplace = True)
        return df

    def Missing_Values(df):
        for i in range(0,len(df.columns)):
            i = df.columns[i]
            print(i,'\n\n',df[i].value_counts(),'\n\n',(df[i].value_counts()).sum(),'\n\n')
        return Missing_Values
    
    def EDA_target(df,Y):
        DF = pandas.DataFrame(Y.value_counts())
        DF['Contribution'] = round(((DF['class'])/len(df)*100),2)
        Missing_values = len(df) - (Y.value_counts()).sum()
        print("Total Missing Values = ", Missing_values, "Data Points, Contributing to ",
              round(((Missing_values/len(df))*100),2),"%")
        return DF
    
    def univariate_plots(Source):
        a = pandas.Series(Source.select_dtypes(include=['int32','int64']).columns)
        leng = len(a)
        for j in range(0,len(a)):
            f, axes = matplot.subplots(1, 2, figsize=(10, 10))
            sns.boxplot(Source[a[j]], ax = axes[0])
            sns.distplot(Source[a[j]], ax = axes[1])
            matplot.subplots_adjust(top =  1.5, right = 10, left = 8, bottom = 1)

        a = pandas.Series(Source.select_dtypes(include=['float64']).columns)
        leng = len(a)
        for j in range(0,len(a)):
            matplot.Text('Figure for float64')
            f, axes = matplot.subplots(1, 2, figsize=(10, 10))
            sns.boxplot(Source[a[j]], ax = axes[0])
            sns.distplot(Source[a[j]], ax = axes[1])
            matplot.subplots_adjust(top =  1.5, right = 10, left = 8, bottom = 1)

        a = pandas.Series(Source.select_dtypes(include=['object']).columns)
        leng = len(a)
        for j in range(0,len(a)):
            matplot.subplots()
            sns.countplot(Source[a[j]])

class Evaluate_Model():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
    
    
    def plot_confusion_matrix(Y_test,Y_predict, target_names,title='Confusion matrix',cmap=None,normalize=True):
        cm = metrics.confusion_matrix(Y_test, Y_predict)
        accuracy = numpy.trace(cm) / float(numpy.sum(cm))
        misclass = 1 - accuracy

        if cmap is None:
            cmap = matplot.get_cmap('Blues')

        matplot.figure(figsize=(8, 6))
        matplot.imshow(cm, interpolation='nearest', cmap=cmap)
        matplot.title(title)
        matplot.colorbar()

        if target_names is not None:
            tick_marks = numpy.arange(len(target_names))
            matplot.xticks(tick_marks, target_names, rotation=45)
            matplot.yticks(tick_marks, target_names)

        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, numpy.newaxis]


        thresh = cm.max() / 1.5 if normalize else cm.max() / 2
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            if normalize:
                matplot.text(j, i, "{:0.4f}".format(cm[i, j]),
                         horizontalalignment="center",
                         color="white" if cm[i, j] > thresh else "black")
            else:
                matplot.text(j, i, "{:,}".format(cm[i, j]),
                         horizontalalignment="center",
                         color="white" if cm[i, j] > thresh else "black")


        matplot.tight_layout()
        matplot.ylabel('True label')
        matplot.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
        matplot.show()
        print(metrics.classification_report(Y_test, Y_predict))
        model_performance = [metrics.accuracy_score(Y_test, Y_predict),metrics.recall_score(Y_test, Y_predict),
                             metrics.precision_score(Y_test, Y_predict),metrics.f1_score(Y_test, Y_predict) ]
        accuracy_report = pandas.DataFrame(model_performance, columns=['Model_Performance'], 
                                       index=['Accuracy','Recall','Precision','f1_Score'])
        return accuracy_report
    

class Build_Model():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        
    def GS_DT(X_train, X_test, Y_train, Y_test,max_depth_start,max_depth_end,max_depth_jump,max_leaf_nodes_start,
          max_leaf_nodes_end,max_leaf_nodes_jump, cv_count):
        max_depth = numpy.arange(max_depth_start,max_depth_end,max_depth_jump)
        max_leaf_nodes = numpy.arange(max_leaf_nodes_start,max_leaf_nodes_end,max_leaf_nodes_jump)
        dt_Model = DecisionTreeClassifier(criterion = 'entropy', random_state=RandomState)
        parameters = {'max_depth': max_depth, 'max_leaf_nodes':max_leaf_nodes}
        dt_GS = GridSearchCV(dt_Model,parameters,cv=cv_count)
        Y_train_1d = (Y_train.values).reshape(((Y_train.shape)[0]),)
        dt_GS.fit(X_train,Y_train_1d)
        dt_GS.predict(X_test)
        depth = (pandas.Series(dt_GS.best_params_))[0]
        leaf_node = (pandas.Series(dt_GS.best_params_))[1]
        dt_model_reg = DecisionTreeClassifier(criterion = 'entropy', max_depth = depth, max_leaf_nodes = leaf_node, random_state= RandomState )
        print(dt_model_reg)
        dt_model_reg.fit(X_train, Y_train)
        Tree_File = open('Decision_Tree.dot','w')
        dot_data = tree.export_graphviz(dt_model_reg, out_file=Tree_File, feature_names = list(X_train), class_names = list(target_names))
        Tree_File.close()
        print (pandas.DataFrame(dt_model_reg.feature_importances_, columns = ["Imp"], index = X_train.columns))
        Y_predict = dt_model_reg.predict(X_test)
        dt_model_reg = plot_confusion_matrix(Y_test,Y_predict, target_names,title='Confusion matrix',cmap=None,normalize=True)
        dt_model_reg = dt_model_reg.rename(columns={"Model_Performance" : "Decision Tree Regularized" })
        dt_model_reg
        return depth, leaf_node, dt_GS.get_params,dt_model_reg

    def Log_model(X_train, X_test, Y_train, Y_test):
        logmodel = LogisticRegression()
        Log_model = logmodel.fit(X_train,Y_train)
        Y_predict = logmodel.predict(X_test)
        Log_model_scores = plot_confusion_matrix(Y_test,Y_predict, target_names,title='Confusion matrix',cmap=None,normalize=True)
        Log_model_scores = Log_model_scores.rename(columns={"Model_Performance" : "Logistic Regression" })
        Log_model_scores
        return Log_model_scores

    def Gau_NB_model(X_train, X_test, Y_train, Y_test):
        NBmodel = GaussianNB()
        Gau_NB_model = NBmodel.fit(X_train,Y_train)
        Y_predict = NBmodel.predict(X_test)
        Gau_NB_model_scores = plot_confusion_matrix(Y_test,Y_predict, target_names,title='Confusion matrix',cmap=None,normalize=True)
        Gau_NB_model_scores = Gau_NB_model_scores.rename(columns={"Model_Performance" : "Gaussian NB" })
        Gau_NB_model_scores
        return Gau_NB_model_scores

    def GS_KNN(X_train,Y_train,X_test,Y_test, n_neighbors_start, n_neighbors_end, n_neighbors_jump,cv_count):
        X_train = preprocessing.scale(X_train)
        X_test = preprocessing.scale(X_test)
        k = numpy.arange(n_neighbors_start, n_neighbors_end, n_neighbors_jump)
        KNN = KNeighborsClassifier()
        parameters = {'n_neighbors': k}
        GS_KNN = GridSearchCV(KNN,parameters,cv=cv_count)
        Y_train_1d = (Y_train.values).reshape(((Y_train.shape)[0]),)
        GS_KNN.fit(X_train,Y_train_1d)
        GS_KNN.predict(X_test)
        n_neighbors = (pandas.Series(GS_KNN.best_params_))[0]
        KNN_model = KNeighborsClassifier(n_neighbors = n_neighbors)
        print(KNN_model)
        KNN_model.fit(X_train, Y_train)
        Y_predict = KNN_model.predict(X_test)
        KNN_model_Scores = plot_confusion_matrix(Y_test,Y_predict, target_names,title='Confusion matrix',cmap=None,normalize=True)
        KNN_model_Scores = KNN_model_Scores.rename(columns={"Model_Performance" : "K Nearest Neighbors" })
        KNN_model_Scores
        return n_neighbors, GS_KNN.get_params,KNN_model_Scores

    def GS_RandomForest(X_train,Y_train,X_test,Y_test, n_estimator_start, n_estimator_end, n_estimator_jump,cv_count):
        n = numpy.arange(n_estimator_start, n_estimator_end, n_estimator_jump)
        RF = RandomForestClassifier(random_state=RandomState)
        parameters = {'n_estimators': n}
        GS_RandomForest = GridSearchCV(RF,parameters,cv=cv_count)
        Y_train_1d = (Y_train.values).reshape(((Y_train.shape)[0]),)
        GS_RandomForest.fit(X_train,Y_train_1d)
        GS_RandomForest.predict(X_test)
        n_estimator = (pandas.Series(GS_RandomForest.best_params_))[0]
        RF = RandomForestClassifier(n_estimators = n_estimator, random_state=RandomState)
        print(RF)
        RF.fit(X_train, Y_train)
        Y_predict = RF.predict(X_test)
        RF_Scores = plot_confusion_matrix(Y_test,Y_predict, target_names,title='Confusion matrix',cmap=None,normalize=True)
        RF_Scores = RF_Scores.rename(columns={"Model_Performance" : "Random Forest" })
        RF_Scores
        return n_estimator, GS_RandomForest.get_params,RF_Scores